### Import python packages

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual, AppLayout, Button, HBox, Layout, VBox, HTML
from IPython.display import display
import ipywidgets as widgets
from dataset import Dataset
import IPython
import ipyspin

### Define the spin for long tasks

In [4]:
s = ipyspin.Spinner()

s.layout.height = '40px'
s.layout.width = '40px'

s.lines = 13  # The number of lines to draw
s.length = 60  # The length of each line
s.width = 8  # The line thickness
s.radius = 8  # The radius of the inner circle
s.scale = 1  # Scales overall size of the spinner
s.corners = 1  # Corner roundness (0..1)
s.speed = 1  # Rounds per second
s.rotate = 0  # The rotation offset
s.animation = 'spinner-line-fade-quick'  # The CSS animation name for the lines
s.direction = 1  # 1: clockwise, -1: counterclockwise
s.color = '#000000'  # CSS color or array of colors
s.fade_color = 'transparent'  # CSS color or array of colors
s.top = '50%'  # Top position relative to parent
s.left = '50%'  # Left position relative to parent
s.shadow = '0 0 1px transparent'  # Box-shadow for the lines
s.z_index = 2000000000  # The z-index (defaults to 2e9)
s.class_name = 'spinner'  # The CSS class to assign to the spinner
s.position = 'absolute'  # Element positioning

s.stop()  # Removes the UI elements from the DOM and stops the animation

s.spin()  # Stopped spinners may be reused by calling spin() again

s  # Displays the spinner

Spinner(class_name='spinner', layout=Layout(height='40px', width='40px'), length=60.0, radius=8.0, width=8.0)

### Select the dataset

Please, see the files included in the folder data. Each of them corresponds to a dataset, including the data criteria and the results of the assessment. Please, remove the # character of the dataset that you want to use.

In [20]:
#file = "data/zeri-data-quality.ttl"
file = "data/bnf-data-quality.ttl"
#file = "data/bne-data-quality.ttl"
#file = "data/ldfi-data-quality.ttl"
dataset = Dataset(file)

In [22]:
sparqlWidget = widgets.Text(
    value=dataset.getEndpoint(),
    placeholder='SPARQL endpoint',
    description='SPARQL:',
    disabled=False,
    layout = widgets.Layout(width='350px')
)

titleWidget = widgets.Text(
    value=dataset.getEndpoint(),
    placeholder='Title',
    description='Title:',
    disabled=False,
    layout = widgets.Layout(width='350px')
)

descriptionWidget = widgets.Text(
    value=dataset.getDescription(),
    placeholder='Description',
    description='Description:',
    disabled=False,
    layout = widgets.Layout(width='350px')
)

introductionLayout = VBox()
introductionItems = []
introductionItems.append(sparqlWidget)
introductionItems.append(titleWidget)
introductionItems.append(descriptionWidget)
introductionLayout.children = introductionItems

### Let's retrieve the metrics

In [25]:
def setCriterion(criterion):
    out = widgets.Output()
    display(out)
    
    with out:
        display(s)
        try:
            result = dataset.runCriterion(criterion)
            for d in result:
                assessmentItems = []
    
                if criterion == "Syntactic validity":
                    assessmentItems.append(widgets.HTML(
                        value='Please, assess the URIs provided in the section Raw Result and using the \
                        <a style="color:blue;" href="https://www.w3.org/RDF/Validator/">W3C RDF Validator service</a>',
                        placeholder='Note',
                        description='Note'
                    ))
            
                assessmentItems.append(widgets.HTML(
                    value=d["label"],
                    placeholder='Label',
                    description='Label'
                ))
                assessmentItems.append(widgets.HTML(
                    value=d["sparqlResult"],
                    placeholder='Result',
                    description='Result'
                ))
                assessmentItems.append(widgets.HTML(
                    value=d["time"],
                    placeholder='Seconds',
                    description='Seconds'
                ))
                
                assessmentLayout = VBox()
                assessmentLayout.children = assessmentItems
        
                children = []
                children.append(assessmentLayout)
                children.append(widgets.Textarea(
                               value=d["query"],
                               placeholder='Query',
                               description='Query:',
                               layout=widgets.Layout(width='50%'),
                               disabled=False))
                    
                children.append(widgets.Textarea(
                               value=d["sparqlResultRaw"],
                               placeholder='Result',
                               description='Result:',
                               layout=widgets.Layout(width='90%'),
                               disabled=False))
                
                accordion = widgets.Accordion(children)
                accordion.set_title(0, 'Assessment')
                accordion.set_title(1, 'SPARQL')
                accordion.set_title(2, 'Raw result')
                
                out.clear_output()
                display(accordion)
        except Exception as e:
            # time out exception
            out.clear_output()
            print(e)
            #display(e)

criteriaWidget = widgets.interactive(setCriterion, criterion=dataset.getCriteria())
criteriaLayout = VBox()
criteriaItems = []
criteriaItems.append(widgets.HTML(
                    value="<b>This section enables users to run the data quality criteria assessment. \
                    First the user selects a data quality criterion. Then, the SPARQL query defined \
                    to assess the data quality criterion is retrieved and executed using the SPARQL endpoint. \
                    Finally, the results are shown including the time taken and the SPARQL query used.</b>"))
criteriaItems.append(criteriaWidget)
criteriaLayout.children = criteriaItems

### Let's retrieve the assessment results

In [28]:
assessmentResults = []

assessmentResults.append(widgets.HTML(
                    value="<b>This section shows the data quality assessment results obtained for the LOD repositories selected. \
                              The results are retrieved from the ttl files in the data folder using the properties dqv:value.</b>"))

# adding the stules for the headers
assessmentResults.append(widgets.HTML(
                    value="<style>.mytext > .widget-label {font-style: italic;color: blue;font-size: 30px;}\
                                  .mytext > input[type='text'] { font-weight: bold; color: black;}</style>"))

assessmentCriterion = []    

tDimension = widgets.Text(value="Dimension")
tDimension.add_class("mytext")

tCriterion = widgets.Text(value="Criterion")
tCriterion.add_class("mytext")

tResult = widgets.Text(value="Result")
tResult.add_class("mytext")

assessmentCriterion.append(tDimension)
assessmentCriterion.append(tCriterion)
assessmentCriterion.append(tResult)

assessmentCriterionLayout = HBox()
assessmentCriterionLayout.children = assessmentCriterion
assessmentResults.append(assessmentCriterionLayout)

for a in dataset.getAssessmentResults():
    assessmentCriterion = []    
    assessmentCriterion.append(widgets.Text(
        value=a["dimension"]
    ))
    assessmentCriterion.append(widgets.Text(
        value=a["criterion"]
    ))
    assessmentCriterion.append(widgets.Text(
        value=a["value"]
    ))
    
    assessmentCriterionLayout = HBox()
    assessmentCriterionLayout.children = assessmentCriterion
    assessmentResults.append(assessmentCriterionLayout)
    
assessmentResultsLayout = VBox()
assessmentResultsLayout.children = assessmentResults

### Create visual interface

In [31]:
children = []
titles = []
tab = widgets.Tab()
children.insert(0,introductionLayout)
children.insert(1,criteriaLayout)
children.insert(2,assessmentResultsLayout)
tab.children = children

titles.insert(0, 'Repository')
titles.insert(1, 'Data quality criteria')
titles.insert(2, 'Assessment results')
tab.titles = titles

tab